In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
def join_content(item):
    content = [it.strip().replace('  ', ' ') for it in item.text.split('\n') if it.strip() != '' ]
    return content

In [4]:
def clean_tags_for_schema_and_snippet(item):
    fonts = item.find_all('font')
    for font in fonts:
        if font is not None:
            font.extract()
    ol = item.find('ol')
    ol.extract()
    ps = item.find_all('p')
    for p in ps:
        p.extract()
        
    return item

In [5]:
def get_schema_and_snippet_texts(item):
    item = clean_tags_for_schema_and_snippet(item)
    content = join_content(item)
    
    texts = (' ').join(content).split('Trecho:')
    schema = texts[0].replace('  ', ' ').strip()
    snippet = texts[1].replace('  ', ' ').strip()
    if not schema[-1] in ['.', '!', '?']:
        schema += '.'
    if snippet[-1] != schema[-1] and schema[:-1].split()[-1] == snippet.split()[-1]:
        snippet += schema[-1]
        
    return schema, snippet

In [6]:
still_in_english = [
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the spot under the tree was hot.",
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the dog was cooler.",
    "I couldn't put the pot on the shelf because the shelf was too tall.",
    "I couldn't put the pot on the shelf because the pot was too high.",
    "There is a pillar between me and the stage, and I can't see around the stage.",
    "There is a pillar between me and the stage, and I can't see the pillar.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear the subway.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear over the announcement.",
    "The large ball crashed right through the table because the table was made of steel.",
    "The large ball crashed right through the table because the large ball was made of styrofoam."
]

In [7]:
subs_not_working = [
    'Jim fez um sinal para o barman e apontou para sua taça vazia.',
    'Jim fez um sinal para o barman e apontou para jim chave do banheiro.',
    'Dan ficou no banco de trás enquanto Bill reinvidicava a frente porque Bill "Eu primeiro!" era lento.',
    'Dan ficou no banco de trás enquanto Bill reinvidicava a frente porque Dan "Eu primeiro!" era mais rápido.',
    'Tom disse "Check" para Ralph enquanto ele movia Ralph bispo.',
    'Tom disse "Check" para Ralph enquanto ele tomava Tom bispo.',
    'John contratou Bill para cuidar Bill.',
    'John foi contratado por Bill para cuidar John.',
    'Ele os abaixou quando o vento parou.',
    'Ele os abriu quando o vento parou.',
    'As raposas estão entrando à noite e atacando as galinhas. Eu terei que matá- las.',
    'As raposas estão entrando à noite e atacando as galinhas. Eu terei que protegê- las.',
    'O caminho até o lago estava interditado, então nós não podíamos usá-o lago.',
    'O caminho até o lago estava interditado, então nós não podíamos alcançá-o caminho.',
    'Sara pegou um livro emprestado da biblioteca porque ela precisa dele para um artigo no qual ela está trabalhando. Ela o artigo lê quando chega do trabalho.',
    'Sara pegou um livro emprestado da biblioteca porque ela precisa dele para um artigo no qual ela está trabalhando. Ela o livro escreve quando chega do trabalho.',
    'Como estava chovendo, eu carreguei a revista na minha mochila para mantê-a mochila seca.',
    'Como estava chovendo, eu carreguei a revista sobre a minha mochila para mantê-a revista seca.',
    'Mary sacou sua flauta e tocou uma de suas peças favoritas. Ela a peça tem desde criança.',
    'Mary sacou sua flauta e tocou uma de suas peças favoritas. Ela a flauta ama desde criança.',
    'Joe vendeu sua casa e comprou uma nova a alguns quilômetros de distância. Ele se mudará dela na quinta-feira.',
    'Durante a tempestade, o poste caiu e atravessou o telhado da minha casa. Agora, eu tenho que mandar removêo telhado.',
    'Durante a tempestade, o poste caiu e atravessou o telhado da minha casa. Agora, eu tenho que mandar consertáo posto.',
    'Eu estava tentando abrir o cadeado com a chave, mas alguém havia preenchido a fechadura com goma de mascar, e eu não conseguia inseri-la .',
    'Eu estava tentando abrir o cadeado com a chave, mas alguém havia preenchido a fechadura com goma de mascar, e eu não conseguia removê-la.',
    'Há uma fenda na parede. É possível enxergar o jardim através a parede.',
    'Há uma fenda na parede. É possível enxergar o jardim atrás a fenda.',
    # ok mas estranho:
    'Eu usei um pano velho para limpar o alicate, e então o alicate coloquei no lixo.',
    'Eu usei um pano velho para limpar o alicate, e então o pano coloquei na gaveta.',
'    Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós os alunos mais jovens punimos.',
    'Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós os alunos mais velhos protegemos.'
    # "policia estavam", errado
    'Os bombeiros chegaram depois da polícia porque a polícia estavam vindo de muito longe.',
    'Os bombeiros chegaram antes da polícia porque os bombeiros estavam vindo de muito longe.',
    'Alongando suas costas, a mulher sorriu para a menina.',
    'Dando um tapinha em suas costas, a mulher sorriu para a menina.',
    # verificar:
    'Quando Tommy derrubou seu sorvete, Timmy gargalhou, então seu pai olhou-o com olhar empático.',
    'Quando Tommy derrubou seu sorvete, Timmy gargalhou, então seu pai olhou-o com olhar severo.'
    # TODO revisar se nao tem mais no PR
]

In [48]:
# def generate_df():
with open('data/processed/port_wsc.html', 'r') as f:
    soup = BeautifulSoup(f, 'html5lib')

rows = []
for item in soup.find('ol').find_all('li', recursive=False):
    skip = False
    for english_sentence in still_in_english:
        if english_sentence[:20] in item.text:
            skip = True
    if skip:
        continue
    for not_working_sentence in subs_not_working:
        if not_working_sentence[:20] in item.text:
            skip = True
    if skip:
        continue

    pronoun = item.find('b').text.strip()
    
    content = join_content(item)
    correct_answer = content[-1].replace('Resposta Correta:', '').strip()[0]
    substitution_a = content[-3]
    substitution_b = content[-2]
    
    schema, snippet = get_schema_and_snippet_texts(item)
    
    row = [schema, snippet, pronoun, correct_answer, substitution_a, substitution_b]
    rows.append(row)

df = pd.DataFrame(rows, columns=['schema', 'snippet', 'pronoun', 'correct_answer', 'substitution_a', 'substitution_b'])
# return df

In [49]:
import re

def generate_full_sentences(row):
    if row.pronoun.islower() and (row.substitution_b.lower() in row.schema or 
                                  row.substitution_b.lower() not in row.schema.lower()):
        subs_a = row.substitution_a.lower()
        subs_b = row.substitution_b.lower()
    else:
        subs_a = row.substitution_a
        subs_b = row.substitution_b

    new_snippet_a = re.sub(r"\b%s\b" % row.pronoun , subs_a, row.snippet)
    new_snippet_b = re.sub(r"\b%s\b" % row.pronoun , subs_b, row.snippet)
    new_schema_a = row.schema.replace(row.snippet, new_snippet_a).strip()
    new_schema_b = row.schema.replace(row.snippet, new_snippet_b).strip()
    
    if row.correct_answer.lower() == 'a':
        return new_schema_a, new_schema_b
    return new_schema_b, new_schema_a

df['correct_sentence'], df['incorrect_sentence'] = zip(*df.apply(generate_full_sentences, axis=1))

In [10]:
# faz sentido se "Humanos pré-históricos" nunca apareceu no texto?

# fred e alice em minúsculo
# filha da mary em minúsculo
# Eu não sei quem jim era --> jim em minusculo
# O golpista maiúsculo
# O tio do Joe maiúsculo
# desenho da tina minúsculo
# fred tinha doze anos de idade

# for index, row in df.iterrows():
#     print(row.incorrect_sentence)

In [11]:
import torch
from nltk.tokenize import word_tokenize

from src.main import setup_torch, get_corpus
from src.utils import get_latest_model_file
from src.winograd_schema_challenge import analyse_single_wsc

In [12]:
setup_torch()
device = torch.device("cuda")
corpus = get_corpus()
ntokens = len(corpus.dictionary)

# TODO remove these two lines
assert ntokens == 602755
assert corpus.valid.size()[0] == 11606861
assert corpus.train.max() < ntokens
assert corpus.valid.max() < ntokens
assert corpus.test.max() < ntokens

In [13]:
model_file_name = get_latest_model_file()

In [50]:
def find_missing_wsc_words_in_corpus_vocab(df, corpus):
    correct_sentences_vocab = [ word for sentence in df.correct_sentence.values 
                               for word in word_tokenize(sentence, language='portuguese')]
    incorrect_sentences_vocab = [ word for sentence in df.incorrect_sentence.values 
                                 for word in word_tokenize(sentence, language='portuguese')]
    wsc_vocab = set(correct_sentences_vocab + incorrect_sentences_vocab)
    # TODO quotes need to be in the same format as wiki corpus, and split from words
    missing_words = []
    for word in wsc_vocab:
        if word not in corpus.dictionary.word2idx:
            missing_words.append(word)
            
    return missing_words

In [51]:
find_missing_wsc_words_in_corpus_vocab(df, corpus)

['Xenophanes',
 'largá-los',
 'retirei',
 'removi',
 'Arqueologistas',
 'reconfortou',
 'assoviando',
 'acudi-lo',
 'mary',
 'fred',
 'enfiei']

In [62]:
# for item in 
df[df.correct_sentence.str.contains('mary')].correct_sentence
# :
#     print(item)

112    Mary acomodou sua filha Anne na cama, para que...
113    Mary acomodou sua filha Anne na cama, para que...
Name: correct_sentence, dtype: object

In [63]:
df.iloc[114].correct_sentence = 'Fred e Alice tinham casacos muito quentes, mas Fred e Alice não estavam preparados para o frio do Alasca.'
df.iloc[115].incorrect_sentence = 'Fred e Alice tinham casacos muito quentes, mas Fred e Alice não eram suficientes para o frio do Alasca.'
df.iloc[132].correct_sentence = 'Fred é a única pessoa viva que lembra de meu pai enquanto criança. Quando Fred viu meu pai pela primeira vez, Fred tinha doze anos de idade.'
df.iloc[133].incorrect_sentence = 'Fred é a única pessoa viva que lembra de meu pai enquanto criança. Quando Fred viu meu pai pela primeira vez, Fred tinha doze meses de idade.'
df.iloc[112].correct_sentence = 'Mary acomodou sua filha Anne na cama, para que Mary pudesse trabalhar.'
df.iloc[112].incorrect_sentence = 'Mary acomodou sua filha Anne na cama, para que a filha da Mary pudesse trabalhar.'
df.iloc[113].correct_sentence = 'Mary acomodou sua filha Anne na cama, para que a filha da Mary pudesse dormir.'
df.iloc[113].incorrect_sentence = 'Mary acomodou sua filha Anne na cama, para que Mary pudesse dormir.'

In [64]:
find_missing_wsc_words_in_corpus_vocab(df, corpus)

['Xenophanes',
 'largá-los',
 'retirei',
 'removi',
 'Arqueologistas',
 'reconfortou',
 'assoviando',
 'acudi-lo',
 'enfiei']

In [65]:
missing_words = find_missing_wsc_words_in_corpus_vocab(df, corpus)

def sentence_to_word_list(sentence):
    word_list = word_tokenize(sentence, language='portuguese')
    word_list = [ word if word not in missing_words else '<UNK>' for word in word_list]
    
    return word_list

def run_test(row):
    winograd_sentences = ((' ').join(sentence_to_word_list(row.correct_sentence)).strip(),
                          (' ').join(sentence_to_word_list(row.incorrect_sentence)).strip())
    return analyse_single_wsc(model_file_name, corpus, ntokens, device, winograd_sentences[0], winograd_sentences[1])

df['test_result'] = df.apply(run_test, axis=1)

accuracy = sum(df.test_result) / len(df)
print('Acurácia: {} para teste realizado com {} exemplos'.format(accuracy, len(df)))

TypeError: must be str, not float

In [66]:
print('Acurácia: {} para teste realizado com {} exemplos'.format(accuracy, len(df)))

Acurácia: 0.8100558659217877 para teste realizado com 179 exemplos


In [74]:
len(df[~df.test_result])

34

In [69]:
len(df[df.test_result])

145

In [70]:
df

,schema,snippet,pronoun,correct_answer,substitution_a,substitution_b,correct_sentence,incorrect_sentence,test_result
0,Os vereadores recusaram a autorização aos mani...,eles temiam a violência.,eles,A,Os vereadores,Os manifestantes,Os vereadores recusaram a autorização aos mani...,Os vereadores recusaram a autorização aos mani...,False
1,Os vereadores recusaram a autorização aos mani...,eles eram favoráveis à violência.,eles,B,Os vereadores,Os manifestantes,Os vereadores recusaram a autorização aos mani...,Os vereadores recusaram a autorização aos mani...,True
2,A medalha não cabe na maleta porque ela é muit...,ela é muito grande.,ela,A,a medalha,a maleta,A medalha não cabe na maleta porque a medalha ...,A medalha não cabe na maleta porque a maleta é...,True
3,A medalha não cabe na maleta porque ela é muit...,ela é muito pequena.,ela,B,a medalha,a maleta,A medalha não cabe na maleta porque a maleta é...,A medalha não cabe na maleta porque a medalha ...,False
4,Joan certificou-se de agradecer Susan por toda...,ela havia recebido.,ela,A,Joan,Susan,Joan certificou-se de agradecer Susan por toda...,Joan certificou-se de agradecer Susan por toda...,True
5,Joan certificou-se de agradecer Susan por toda...,ela havia oferecido.,ela,A,Joan,Susan,Joan certificou-se de agradecer Susan por toda...,Joan certificou-se de agradecer Susan por toda...,True
6,"Paul tentou ligar para o George, mas ele não f...",ele não foi bem sucedido.,ele,A,Paul,George,"Paul tentou ligar para o George, mas Paul não ...","Paul tentou ligar para o George, mas George nã...",True
7,"Paul tentou ligar para o George, mas ele não e...",ele não estava disponível.,ele,B,Paul,George,"Paul tentou ligar para o George, mas George nã...","Paul tentou ligar para o George, mas Paul não ...",False
8,"O advogado fez uma pergunta ao juiz, mas ele e...",ele estava relutante,ele,A,O advogado,O juiz,"O advogado fez uma pergunta ao juiz, mas o adv...","O advogado fez uma pergunta ao juiz, mas o jui...",False
9,"O advogado fez uma pergunta ao juiz, mas ele e...",ele estava relutante,ele,B,O advogado,O juiz,"O advogado fez uma pergunta ao juiz, mas o jui...","O advogado fez uma pergunta ao juiz, mas o adv...",True


In [72]:
if analyse_single_wsc(model_file_name, corpus, ntokens, device,
                      'A medalha não cabe na maleta porque a maleta é pequena .',
                      'A medalha não cabe na maleta porque a medalha é pequena .'):
    print('Right choice :D')
else:
    print('Wrong :(')

Wrong :(


In [73]:
145/283

0.5123674911660777